# Librerias

In [28]:
import yfinance as yf
from datetime import datetime

import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM

import pandas as pd

# Descargar datos historicos de los últimos 20 años

In [29]:
# Importar la clase datetime del módulo datetime
from datetime import datetime

# Obtener la fecha y hora actuales
fecha_final = datetime.now()

# Calcular la fecha de inicio hace 20 años a partir de la fecha actual
fecha_inicio = datetime(fecha_final.year-20, fecha_final.month, fecha_final.day)

# Definir el símbolo de la acción que se desea descargar (en este caso, "GOOG" para Google)
simbolo_accion = "GOOG"

# Utilizar la función download de yfinance (yf) para obtener los datos históricos de la acción "GOOG"
# entre las fechas de inicio y final
datos_google = yf.download(simbolo_accion, fecha_inicio, fecha_final)


[*********************100%%**********************]  1 of 1 completed


## Visualizar datos descargados

In [30]:
datos_google.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2004-08-19,2.490664,2.591785,2.390042,2.499133,2.499133,897427216
2004-08-20,2.515820,2.716817,2.503118,2.697639,2.697639,458857488
2004-08-23,2.758411,2.826406,2.716070,2.724787,2.724787,366857939
2004-08-24,2.770615,2.779581,2.579581,2.611960,2.611960,306396159
2004-08-25,2.614201,2.689918,2.587302,2.640104,2.640104,184645512


# Extraer los datos de cierre

In [31]:

precio_cierre_ajustado = datos_google[['Adj Close']]

## Visualizar los datos extraídos

In [32]:
precio_cierre_ajustado.head()

,Adj Close
Date,
2004-08-19,2.499133
2004-08-20,2.697639
2004-08-23,2.724787
2004-08-24,2.611960
2004-08-25,2.640104


# Escalamiento de los precios ajustados al cierre entre 0 y 1

In [33]:

# Crear un objeto escalador MinMaxScaler que escalará los datos al rango de 0 a 1
escalador = MinMaxScaler(feature_range=(0,1))

# Utilizar el escalador para transformar los datos de precio de cierre ajustado
datos_escalados = escalador.fit_transform(precio_cierre_ajustado)


In [34]:
# Imprimir los datos escalados
print(datos_escalados)

# Imprimir la longitud de los datos escalados (número de filas)
print(len(datos_escalados))


[[4.80154176e-05]
 [1.20751854e-03]
 [1.36609420e-03]
 ...
 [9.80140028e-01]
 [9.83703134e-01]
 [9.89719514e-01]]
4968


# Preparar datos de entrenamiento

In [35]:
# Lista para almacenar los datos de entrada (x) y de salida (y)
datos_x = []
datos_y = []

# Crear datos de entrenamiento (x) y de salida (y) con una ventana de tiempo de 100 días
for i in range(100, len(datos_escalados)):
    # Agregar a datos_x una ventana de 100 días de datos escalados, desde i-100 hasta i
    datos_x.append(datos_escalados[i-100:i])
    # Agregar a datos_y el siguiente dato escalado después de la ventana de 100 días
    datos_y.append(datos_escalados[i])

# Convertir las listas a arrays de numpy
datos_x, datos_y = np.array(datos_x), np.array(datos_y)

# Definir la longitud para dividir los datos en entrenamiento y prueba (70% entrenamiento, 30% prueba)
longitud_division = int(len(datos_x) * 0.7)

# Dividir los datos en conjuntos de entrenamiento y prueba
x_entrenamiento = datos_x[:longitud_division]
y_entrenamiento = datos_y[:longitud_division]

x_prueba = datos_x[longitud_division:]
y_prueba = datos_y[longitud_division:]


# Construir el modelo

In [36]:

# Crear un modelo de red neuronal secuencial
modelo = Sequential()

# Agregar una capa LSTM con 128 neuronas, que recibe una secuencia de entrada y devuelve una secuencia
# La forma de entrada es (número de pasos de tiempo, número de características)
modelo.add(LSTM(128, return_sequences=True, input_shape=(x_entrenamiento.shape[1],1)))

# Agregar otra capa LSTM con 64 neuronas, que recibe una secuencia de entrada y devuelve un solo valor
modelo.add(LSTM(64, return_sequences=False))

# Agregar una capa densa con 25 neuronas
modelo.add(Dense(25))

# Agregar una capa densa de salida con 1 neurona para predecir el siguiente valor
modelo.add(Dense(1))

# Compilar el modelo con el optimizador 'adam' y la función de pérdida 'mean_squared_error'
modelo.compile(optimizer='adam', loss='mean_squared_error')


c:\Users\ORPE&AGAL\Desktop\IA\projectIA\stock_price_prediction-main\vp\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


# Entrenar modelo

In [37]:
# Entrenar el modelo con los datos de entrenamiento
# batch_size=1 indica que se utiliza un tamaño de lote de 1 (se actualiza el modelo después de cada muestra)
# epochs=2 indica que se realizarán 2 épocas de entrenamiento
modelo.fit(x_entrenamiento, y_entrenamiento, batch_size=1, epochs=2)


Epoch 1/2
3407/3407 ━━━━━━━━━━━━━━━━━━━━ 47s 13ms/step - loss: 3.2314e-04
Epoch 2/2
3407/3407 ━━━━━━━━━━━━━━━━━━━━ 46s 14ms/step - loss: 7.2763e-05


# Ver el resumen del modelo

In [38]:
modelo.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 100, 128)       │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 25)             │         1,625 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 352,859 (1.35 MB)

 Trainable params: 117,619 (459.45 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 235,240 (918.91 KB)

# Guardar el modelo 

In [39]:
modelo.save("Stock_prediction_model.keras")